<a href="https://colab.research.google.com/github/hrushikeshsahu19/ML_algorithm_custom_code/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth  # Maximum depth of the tree
        self.tree = None  # The decision tree structure

    def _gini_impurity(self, y):
        """
        Compute the Gini Impurity for a given set of labels.

        Parameters:
        y (numpy.ndarray): Array of labels.

        Returns:
        float: Gini impurity value.
        """
        _, counts = np.unique(y, return_counts=True)
        prob_sq = (counts / len(y)) ** 2
        return 1 - np.sum(prob_sq)

    def _best_split(self, X, y):
        """
        Find the best feature and value to split the data on.

        Parameters:
        X (numpy.ndarray): Feature matrix.
        y (numpy.ndarray): Target vector.

        Returns:
        tuple: Best feature index and value for the split.
        """
        best_gini = float('inf')
        best_split = None
        n_samples, n_features = X.shape

        for feature_idx in range(n_features):
            feature_values = np.unique(X[:, feature_idx])
            for value in feature_values:
                # Split the data based on the feature and value
                left_mask = X[:, feature_idx] <= value
                right_mask = ~left_mask

                left_y, right_y = y[left_mask], y[right_mask]

                # Calculate Gini impurity for the split
                gini = (len(left_y) / n_samples) * self._gini_impurity(left_y) + \
                       (len(right_y) / n_samples) * self._gini_impurity(right_y)

                if gini < best_gini:
                    best_gini = gini
                    best_split = (feature_idx, value)

        return best_split

    def _build_tree(self, X, y, depth=0):
        """
        Recursively build the decision tree.

        Parameters:
        X (numpy.ndarray): Feature matrix.
        y (numpy.ndarray): Target vector.
        depth (int): Current depth of the tree.

        Returns:
        dict: The decision tree structure.
        """
        n_samples, n_features = X.shape
        unique_classes = np.unique(y)

        # If all samples have the same class or max depth is reached, return the class label
        if len(unique_classes) == 1 or (self.max_depth and depth >= self.max_depth):
            return unique_classes[0]

        # Find the best split
        feature_idx, value = self._best_split(X, y)

        if feature_idx is None:
            return np.random.choice(unique_classes)

        # Split the data
        left_mask = X[:, feature_idx] <= value
        right_mask = ~left_mask

        left_tree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_tree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            'feature_idx': feature_idx,
            'value': value,
            'left': left_tree,
            'right': right_tree
        }

    def fit(self, X, y):
        """
        Fit the decision tree model.

        Parameters:
        X (numpy.ndarray): Feature matrix.
        y (numpy.ndarray): Target vector.
        """
        self.tree = self._build_tree(X, y)

    def _predict_single(self, x, tree):
        """
        Predict the class label for a single sample based on the tree.

        Parameters:
        x (numpy.ndarray): Single sample.
        tree (dict): The decision tree structure.

        Returns:
        int: Predicted class label.
        """
        if isinstance(tree, dict):
            feature_value = x[tree['feature_idx']]
            if feature_value <= tree['value']:
                return self._predict_single(x, tree['left'])
            else:
                return self._predict_single(x, tree['right'])
        else:
            return tree

    def predict(self, X):
        """
        Predict class labels for the input data.

        Parameters:
        X (numpy.ndarray): Feature matrix.

        Returns:
        numpy.ndarray: Predicted class labels.
        """
        return np.array([self._predict_single(x, self.tree) for x in X])


# Example usage
if __name__ == "__main__":
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    # Generate a synthetic dataset
    X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the decision tree model
    model = DecisionTree(max_depth=10)
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # Evaluate the model
    print("Accuracy:", accuracy_score(y_test, predictions))


Accuracy: 0.85
